In [1]:
import openai
import pandas as pd
from openai.embeddings_utils import get_embedding,cosine_similarity
import keys

In [211]:
import requests, uuid, json

def translate(text):
    # Add your key and endpoint
    key = "c3cc6068a4664a27ab85c48d39082489"
    endpoint = "https://api.cognitive.microsofttranslator.com/"

    # location, also known as region.
    # required if you're using a multi-service or regional (not global) resource. It can be found in the Azure portal on the Keys and Endpoint page.
    location = "westeurope"

    path = '/translate'
    constructed_url = endpoint + path

    params = {
        'api-version': '3.0',
        'from': 'hu',
        'to': ['en']
    }

    headers = {
        'Ocp-Apim-Subscription-Key': key,
        # location required if you're using a multi-service or regional (not global) resource.
        'Ocp-Apim-Subscription-Region': location,
        'Content-type': 'application/json',
        'X-ClientTraceId': str(uuid.uuid4())
    }

    # You can pass more than one object in body.
    body = [{
        'text': text
    }]

    request = requests.post(constructed_url, params=params, headers=headers, json=body)
    response = request.json()
    
#     return json.dumps(response, sort_keys=True, ensure_ascii=False, indent=4, separators=(',', ': '))
    return response[0]["translations"][0]["text"]

In [2]:
openai.api_type = "azure"
openai.api_key = "c4a1d51a3b974ac6b0ac45b3db2cd7b4"
openai.api_base = "https://cheppy.openai.azure.com/"
openai.api_version = "2023-03-15-preview"

In [111]:
product_data_df = pd.read_json("/media/itqs/10_TB_Hdd/Tamas/chatgpt-product-recommendation-embeddings/data/data.json")

In [112]:
products_available = product_data_df[product_data_df["Availability"]=="in stock"]

In [113]:
products_available.columns

Index(['DisplayText', 'Url', 'ImageUrl', 'PopularityScore', 'Category',
       'Price', 'Identifier', 'Synonyms', 'Description', 'Brand', 'Attributes',
       'UnitPrice', 'OldPrice', 'Availability', 'DiscountStart', 'DiscountEnd',
       'Discount'],
      dtype='object')

In [84]:
products_available['Category'] = products_available['Category'].apply(lambda x: x.replace("/", " "))

/tmp/ipykernel_1307399/1444696639.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  products_available['Category'] = products_available['Category'].apply(lambda x: x.replace("/", " "))


In [114]:
products_available.head(1)

,DisplayText,Url,ImageUrl,PopularityScore,Category,Price,Identifier,Synonyms,Description,Brand,Attributes,UnitPrice,OldPrice,Availability,DiscountStart,DiscountEnd,Discount
2,SANIMIX HIDROMASSZÁZS ZUHANYKABIN 90X90X215 CM...,https://www.praktiker.hu/furdoszoba/szaniter/z...,https://webimg.praktiker.hu/_upload/images/pra...,100,Fürdőszoba/Szaniter/Zuhanykabin,219900.0,339001,None,Hidromasszázs zuhanykabin mérete: <span>90X90X...,Sanimix,"{'Szallithato': True, 'Tartosan_alacsony_ar': ...","219.900,00 Ft / darab",NaN,in stock,,,NaN


In [132]:
products_available['combined'] = products_available.apply(lambda row: f"{row['Category']}, {row['Description']}", axis=1)
products_available.head(1)["combined"].values

array(['Kert/Kerti szerszám/Lombseprű, Állítható, műanyag, 6 fokozatú lombseprű, nyélhossz mérete: 130cm, a seprű szélessége 35-52cm között állítható.'],
      dtype=object)

In [153]:
products_available.combined

4402    Fürdőszoba/Fürdőszoba felszerelés/Zuhanygarnit...
2257    Fürdőszoba/Fürdőszoba felszerelés/Zuhanyfej, <...
4213    Fürdőszoba/Fürdőszoba felszerelés/Zuhanyfej, <...
234     Fürdőszoba/Fürdőszoba felszerelés/Zuhanyfej, <...
4301    Fürdőszoba/Fürdőszoba felszerelés/Zuhanygarnit...
                              ...                        
5080    Lakberendezés, háztartás/Függöny, karnis, árny...
935     Lakberendezés, háztartás/Függöny, karnis, árny...
806     Lakberendezés, háztartás/Világítás, lámpa/Kült...
2204    Műszaki, Gép, Szerszám/Elektromos szerszámgép/...
3013    Műszaki, Gép, Szerszám/Mérőeszköz/Környezetmér...
Name: combined, Length: 1583, dtype: object

In [133]:
products_available['text_embedding'] = products_available.combined.apply(lambda x: get_embedding(x, engine='text-embedding-ada-002'))
products_available.head(1)

,DisplayText,Url,ImageUrl,PopularityScore,Category,Price,Identifier,Synonyms,Description,Brand,Attributes,UnitPrice,OldPrice,Availability,DiscountStart,DiscountEnd,Discount,combined,text_embedding,search_products
841,GARDENA ÁLLÍTHATÓ LOMBSEPRŰ 6 FOKOZATÚ MŰANYAG,https://www.praktiker.hu/kert/kerti-szerszam/l...,https://webimg.praktiker.hu/_upload/images/pra...,100,Kert/Kerti szerszám/Lombseprű,9499.0,301435,10850;ALLITHATO LOMBSEPRU 6 FOKOZATU MUANYAG,"Állítható, műanyag, 6 fokozatú lombseprű, nyél...",Gardena,"{'Szallithato': True, 'Tartosan_alacsony_ar': ...","9.499,00 Ft / darab",8549.0,in stock,2023-05-11,2023-05-22,NaN,"Kert/Kerti szerszám/Lombseprű, Állítható, műan...","[-0.010269951075315475, 0.009866831824183464, ...",0.839417


In [248]:
customer_input = "Tudnál virágládát ajánlani nekem?"
response = openai.Embedding.create(
    input=translate(customer_input),
    engine="text-embedding-ada-002"
)
embeddings_customer_question = response['data'][0]['embedding']

In [249]:
translate(customer_input)

'Can you recommend me a flower box?'

In [250]:
products_available['search_products'] = products_available.text_embedding.apply(lambda x: cosine_similarity(x, embeddings_customer_question))
products_available = products_available.sort_values('search_products', ascending=False)
top_3_products_df = products_available.head(3)
top_3_products_df

,DisplayText,Url,ImageUrl,PopularityScore,Category,Price,Identifier,Synonyms,Description,Brand,Attributes,UnitPrice,OldPrice,Availability,DiscountStart,DiscountEnd,Discount,combined,text_embedding,search_products
2305,KERTIMAG BALKON VIRÁG CSÜNGŐ LOBÉLIA KÉK,https://www.praktiker.hu/kert/noveny/novenymag...,https://webimg.praktiker.hu/_upload/images/pra...,100,Kert/Növény/Növénymag,319.0,402142,None,"Balkonládák, függőkosarak beültetésére ajánlot...",Budapesti kertimag,"{'Szallithato': True, 'Tartosan_alacsony_ar': ...","319,00 Ft / csomag",NaN,in stock,,,NaN,"Kert/Növény/Növénymag, Balkonládák, függőkosar...","[-0.005311526823788881, -0.018064450472593307,...",0.788165
4279,KERTIMAG VÁGOTT VIRÁG ÉVELŐ SZARKALÁB SZÍNKEVERÉK,https://www.praktiker.hu/kert/noveny/novenymag...,https://webimg.praktiker.hu/_upload/images/pra...,100,Kert/Növény/Növénymag,319.0,402402,None,Évelőágyak díszítésére ajánlott. Vágott virágk...,Budapesti kertimag,"{'Szallithato': True, 'Tartosan_alacsony_ar': ...","319,00 Ft / csomag",NaN,in stock,,,NaN,"Kert/Növény/Növénymag, Évelőágyak díszítésére ...","[-0.01398422196507454, -0.01633646711707115, 0...",0.781098
8,JKH BALKONLÁDATARTÓ VAS ZÖLD 2DB-OS 14 X 18 CM,https://www.praktiker.hu/kert/novenyapolas/vir...,https://webimg.praktiker.hu/_upload/images/pra...,100,Kert/Növényápolás/Virágcserép,2399.0,291098,None,"<p>Zöld színű, 14x18 cm-es balkonládatartó vas...",JKH,"{'Szallithato': True, 'Tartosan_alacsony_ar': ...","1.199,50 Ft / darab",NaN,in stock,,,NaN,"Kert/Növényápolás/Virágcserép, <p>Zöld színű, ...","[-0.003715106286108494, -0.0015907941851764917...",0.779099


In [246]:
# message_objects = []
# message_objects.append({"role":"system", "content":"You're a chatbot helping customers with DIY related questions and helping them with product recommendations"})
# message_objects.append({"role":"user", "content": customer_input})
# message_objects.append({"role":"user", "content": f"Please give me a detailed explanation of your recommendations"})
# message_objects.append({"role":"user", "content": "Please be friendly and talk to me like a person, don't just give me a list of recommendations"})
# products_list = []

# for index, row in top_3_products_df.iterrows():
#     brand_dict = {'role': "assistant", "content": f"{row['combined']}"}
#     products_list.append(brand_dict)
# products_list
# message_objects.append({"role": "assistant", "content": f"I found these 3 products I would recommend"})
# message_objects.extend(products_list)
# message_objects.append({"role": "assistant", "content":"Here's my summarized recommendation of products, and why it would suit you:"})
# message_objects

message_objects = []
message_objects.append({"role":"system", "content":"Egy chatbot vagy, aki egy barkácsáruház termékeivel kacsolatban válaszolsz kérdésekre és ajánlásokat adsz."})
message_objects.append({"role":"user", "content": translate(customer_input)})
products_list = []

for index, row in top_3_products_df.iterrows():
    brand_dict = {'role': "assistant", "content": f"{row['combined']}"}
    products_list.append(brand_dict)
products_list
message_objects.append({"role": "assistant", "content": f"Ezt a 3 terméket találtam:"})
message_objects.extend(products_list)
message_objects.append({"role": "assistant", "content":"Itt van az összegzett ajánlásom a termékekről és hogy miért megfelelő:"})
message_objects


[{'role': 'system',
  'content': 'Egy chatbot vagy, aki egy barkácsáruház termékeivel kacsolatban válaszolsz kérdésekre és ajánlásokat adsz.'},
 {'role': 'user', 'content': 'Can you recommend me a shower cabin?'},
 {'role': 'assistant', 'content': 'Ezt a 3 terméket találtam:'},
 {'role': 'assistant',
  'content': 'Fürdőszoba/Szaniter/Zuhanykabin, Hidromasszázs zuhanykabin mérete: <span>90X90X215 cm.</span> Hátfal: fekete biztonsági üveg. 2 tolóajtóval 4mm víztiszta biztonsági üvegből. Görgők: felül dupla, alul szimpla, kinyomós. 2 fix üveg 4mm víztiszta biztonsági üvegből .Profilok: ezüst, alumínium, 4cm. Középmély akril tálca integrált ülőkével. 6 fúvókával. Csaptelep: kerámiabetétes keverőcsap és funkcióváltó. Zuhany: kézi zuhany, esőztető. Világítás: tető és háttérvilágítás a panelen. Szellőztetés: ventilátoros. Vezérlőpanel: elektronikus. Extra: hangszóró és rádió. Tartozék: Piperepolc.'},
 {'role': 'assistant',
  'content': 'Fürdőszoba/Szaniter/Zuhanykabin, <p><span style="font-si

In [247]:
completion = openai.ChatCompletion.create(
  engine="GPT35",
  messages=message_objects
)
message_objects.append({"role": "assistant", "content": completion.choices[0].message['content']})
print(completion.choices[0].message['content'])

Az általad kérdezett zuhanykabinok közül az első két termék alkalmas lehet az igényeidhez. Mindkét zuhanykabin rendelkezik hidromasszázs funkcióval és biztonsági üvegből készült ajtókkal, melyek számos előnnyel járnak a hagyományos üveggel szemben. Az első zuhanykabin esetében a fekete hátfal és a világítás is része az ajánlatnak, valamint a vásárlók értékelik az elektronikus vezérlőpanel, a hangszóró és a rádió meglétét. Azonban, ha inkább az egyszerűbb és elegáns megjelenés a cél, akkor a második zuhanykabin lehet a jobb választás, mivel szögletes és akril zuhanytálcát is tartalmaz. Végül azonban az dönti el, hogy az egyszerűséget vagy a nagyobb funkciókat keresed-e és milyen stílusú zuhanykabint szeretnél.
